# PARCELS Tutorial

Welcome to a quick tutorial on PARCELS. This is meant to get you started with the code, and give you a flavour of some of the key features of PARCELS.

In this tutorial, we will first cover how to run a set of particles [from a very simple idealised grid](#idealised). Then, we will show how to [add custom behaviour](#customkernel) to the particles. And finally, we will show how to [run particles in a set of NetCDF files from external data](#netcdf)

Let's start with importing the relevant modules. The key ones are all in the `parcels` directory. First load in the key functions (don't worry about the `"Matplotlib is building the font cache using fc-list. This may take a moment"` warning that might occur in `matplotlib`)

In [ ]:
%matplotlib inline
from parcels import *
import numpy as np
import math
from datetime import timedelta

## Running particles in an idealised grid <a name="idealised"></a>

The first step to running particles with PARCELS is to define a `grid` object. In this first case, we use a simple flow of two idealised moving eddies. That field is saved in NetCDF format in the directory `examples/MovingEddies_data`. Since we know that the files are in what's called `NEMO` format, we can call these files using the function `Grid.from_nemo`.

In [ ]:
def moving_eddies_grid(xdim=200, ydim=350):
    """Generate a grid encapsulating the flow field consisting of two
    moving eddies, one moving westward and the other moving northwestward.

    Note that this is not a proper geophysical flow. Rather, a Gaussian eddy is moved
    artificially with uniform velocities. Velocities are calculated from geostrophy.
    """
    # Set NEMO grid variables
    depth = np.zeros(1, dtype=np.float32)
    time = np.arange(0., 25. * 86400., 86400., dtype=np.float64)

    # Coordinates of the test grid (on A-grid in deg)
    lon = np.linspace(0, 4, xdim, dtype=np.float32)
    lat = np.linspace(45, 52, ydim, dtype=np.float32)

    # Grid spacing in m
    def cosd(x):
        return math.cos(math.radians(float(x)))
    dx = (lon[1] - lon[0]) * 1852 * 60 * cosd(lat.mean())
    dy = (lat[1] - lat[0]) * 1852 * 60

    # Define arrays U (zonal), V (meridional), W (vertical) and P (sea
    # surface height) all on A-grid
    U = np.zeros((lon.size, lat.size, time.size), dtype=np.float32)
    V = np.zeros((lon.size, lat.size, time.size), dtype=np.float32)
    P = np.zeros((lon.size, lat.size, time.size), dtype=np.float32)

    # Some constants
    corio_0 = 1.e-4  # Coriolis parameter
    h0 = 1  # Max eddy height
    sig = 0.5  # Eddy e-folding decay scale (in degrees)
    g = 10  # Gravitational constant
    eddyspeed = 0.1  # Translational speed in m/s
    dX = eddyspeed * 86400 / dx  # Grid cell movement of eddy max each day
    dY = eddyspeed * 86400 / dy  # Grid cell movement of eddy max each day

    [x, y] = np.mgrid[:lon.size, :lat.size]
    for t in range(time.size):
        hymax_1 = lat.size / 7.
        hxmax_1 = .75 * lon.size - dX * t
        hymax_2 = 3. * lat.size / 7. + dY * t
        hxmax_2 = .75 * lon.size - dX * t

        P[:, :, t] = h0 * np.exp(-(x-hxmax_1)**2/(sig*lon.size/4.)**2-(y-hymax_1)**2/(sig*lat.size/7.)**2)
        P[:, :, t] += h0 * np.exp(-(x-hxmax_2)**2/(sig*lon.size/4.)**2-(y-hymax_2)**2/(sig*lat.size/7.)**2)

        V[:-1, :, t] = -np.diff(P[:, :, t], axis=0) / dx / corio_0 * g
        V[-1, :, t] = V[-2, :, t]  # Fill in the last column

        U[:, :-1, t] = np.diff(P[:, :, t], axis=1) / dy / corio_0 * g
        U[:, -1, t] = U[:, -2, t]  # Fill in the last row

    return Grid.from_data(U, lon, lat, V, lon, lat, depth, time, field_data={'P': P})

In [ ]:
grid = moving_eddies_grid()

The next step is to define a `ParticleSet`, that lives on this grid. In this case, we start 2 particles at (3.3E, 46N) and (3.3E, 47.8N). Note that we use `JITParticle` as `pclass`, because we will be executing the advection in JIT (Just-In-Time) mode. The alternative is to run in `scipy` mode, in which case `pclass` is simply `Particle`

In [ ]:
pset = grid.ParticleSet(size=2,             # the number of particles
                        pclass=JITParticle, # the type of particles (JITParticle or Particle)
                        lon=[ 3.3,  3.3],   # a vector of release longitudes 
                        lat=[46.0, 47.8])   # a vector of release latitudes

Print the `ParticleSet` to see where they start

In [ ]:
print pset

This output shows for each particle the (longitude, latitude, depth), and then in square brackets the grid indices of the longitude and latitude

The final step is to run (or 'execute') the `ParticelSet`. We run the particles using the `AdvectionRK4` kernel, which is a 4th order Runge-Kutte implementation that comes with PARCELS. We run the particles for 6 days (using the `timedelta` function from `datetime`), at an RK4 timestep of 5 minutes. We store the trajectory information at an interval of 1 hour in a file called `EddyParticles.nc`.

In [ ]:
pset.execute(AdvectionRK4,                # the kernel (which defines how particles move)
             runtime=timedelta(days=6),   # the total length of the run
             dt=timedelta(minutes=5),     # the timestep of the kernel
             interval=timedelta(hours=1), # the interval at which output is stored
             output_file=pset.ParticleFile(name="EddyParticles")) # the name of the output file

The code should have run, which can be confirmed by printing the `ParticleSet` again

In [ ]:
print pset

Or by quickly plotting the output file `EddyParticles.nc`.

In [ ]:
run scripts/plotParticles.py -p EddyParticles.nc

Now one of the neat features of PARCELS is that the particles can be plotted as a movie during execution. To rerun the particles while plotting them on top of the zonal velocity field (`grid.U`), first reinitialise the `ParticleSet` and then re-execute. However, now rather than saving the output to a file, display a movie using the `show_movie` keyword, in this case with the zonal velocity `grid.U` as background

In [ ]:
## THIS DOES NOT WORK YET IN THIS IPYTHON NOTEBOOK, BECAUSE OF THE INLINE PLOTTING. 
## IT WILL WORK ON MOST MACHINES, THOUGH
# pset = grid.ParticleSet(size=2, pclass=JITParticle, lon=[3.3, 3.3], lat=[46.0, 47.8])
# pset.execute(AdvectionRK4, 
#              runtime=timedelta(days=6), 
#              dt=timedelta(minutes=5), 
#              interval=timedelta(hours=1),
#              show_movie=grid.U)

## Adding a custom behaviour kernel <a name="customkernel"></a>

A key feature of PARCELS is the ability to quickly create very simple kernels, and add them to the execution. Kernels in this case our little snippets of code that alter the trajectories of the particles. 
In this example, we'll create a simple kernel where particles obtain an extra 5 m/s westward velocity after 1 day. Of course, this is not very realistic scenario, but it nicely illustrates the power of custom kernels.

In [ ]:
def WestVel(particle, grid, time, dt):
    if time > 86400:
        uvel = -2.
        particle.lon += uvel * dt / 1852 / 60

Now reset the `ParticleSet` again, and re-execute. Note that we have now changed `kernel` to be `AdvectionRK4 + k_WestVel`, where `k_WestVel` is the `WestVel` function as defined above cast into a `Kernel` object (via the `pset.Kernel` call). Note also that we run in `scipy` mode now (we have changed the `pclass` in the `ParticleSet` to `Particle`); in general it is recommended to run in `scipy` mode when debugging code.

In [ ]:
pset = grid.ParticleSet(size=2, pclass=Particle, lon=[3.3, 3.3], lat=[46.0, 47.8])

k_WestVel = pset.Kernel(WestVel)       # casting the WestVel function to a kernel object

pset.execute(AdvectionRK4 + k_WestVel, # simply add kernels using the + operator
             runtime=timedelta(days=3), 
             dt=timedelta(minutes=5), 
             interval=timedelta(hours=1),
             output_file=pset.ParticleFile(name="EddyParticles_WestVel"))

And now plot this new particle field

In [ ]:
run scripts/plotParticles.py -p EddyParticles_WestVel.nc